## Oracle AI Data Platform v1.0

Copyright © 2025, Oracle and/or its affiliates.

Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl/

### Example: Read Pipe-Delimited File and Create External Table in Spark

This notebook demonstrates how to read a pipe-delimited (`|`) file from OCI Object Storage using Spark, infer its schema, and create an external table in the Master Catalog.

#### Assumptions:
- The source data is stored in a CSV format in OCI Object Storage.
- default is the catalog used to register the external table.
- Header row is present and schema is inferred automatically.

In [ ]:
catalog_name = "default"
schema_name = "default"
table_name = "sample_pipe_table"

fq_table_name = f"{catalog_name}.{schema_name}.{table_name}"

csv_data_folder_path = "<REPLACE_WITH_YOUR_OCI_CSV_PATH>" ## Example "oci://bucket-name@namespace/foldername/"

csv_read_options = {
    "header": "true",
    "inferSchema": "true",
    "delimiter": "|"
}

csv_df = spark.read.options(**csv_read_options).format("csv").load(csv_data_folder_path)

#csv_df.show()

columns_definitions = csv_df._jdf.schema().toDDL()
#print(columns_definitions)

if spark.catalog.tableExists(fq_table_name):
    spark.sql(f"DROP TABLE IF EXISTS {fq_table_name}").show()
    print(f"Dropped the external table: {fq_table_name}")
else:
    print(f"Didn't drop the external table `{fq_table_name}` as it doesn't exist!")
    
create_table_ddl = f"""CREATE EXTERNAL TABLE IF NOT EXISTS {fq_table_name}
  ({columns_definitions})
  USING CSV
  OPTIONS('header'='true', 'delimiter'='|')
  LOCATION '{csv_data_folder_path}'"""

#print(f"Would create table using below DDL:\n{create_table_ddl}")


if not spark.catalog.tableExists(fq_table_name):
    spark.sql(create_table_ddl).show()
    print(f"Created external table: {fq_table_name}")
else:
    print(f"Didn't create the external table `{fq_table_name}` as it already exists")

In [ ]:
df = spark.read.table("default.default.sample_pipe_table")
df.show()